In [1]:
import cv2
import numpy as np
import os
import random

import os
import random
from PIL import Image
import numpy as np

In [20]:
# def read_yolo_annotations(file_path):
#     annotations = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             annotations.append([float(x) for x in line.strip().split()])
#     return annotations

# def write_yolo_annotations(file_path, annotations, crop_size):
#     with open(file_path, 'w') as file:
#         for ann in annotations:
#             koef = crop_size / FINAL_SIZE
#             ann = [int(ann[0]), koef*ann[1], koef*ann[2], koef*ann[3], koef*ann[4]]
#             file.write(' '.join(map(str, ann)) + '\n')

# def is_object_fully_contained(x, y, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
#     return (x < x_center_abs - width_abs / 2 and
#             x + crop_size > x_center_abs + width_abs / 2 and
#             y < y_center_abs - height_abs / 2 and
#             y + crop_size > y_center_abs + height_abs / 2)

# def is_crop_without_objects(x, y, annotations, w, h, crop_size):
#     for ann in annotations:
#         _, x_center, y_center, width, height = ann
#         x_center_abs = x_center * w
#         y_center_abs = y_center * h
#         width_abs = width * w
#         height_abs = height * h

#         if not (x + crop_size <= x_center_abs - width_abs / 2 or
#                 x >= x_center_abs + width_abs / 2 or
#                 y + crop_size <= y_center_abs - height_abs / 2 or
#                 y >= y_center_abs + height_abs / 2):
#             return False
#     return True

# def resize_annotations(annotations, original_size, target_size):
#     scale_x = target_size[0] / original_size[0]
#     scale_y = target_size[1] / original_size[1]
#     resized_annotations = []
#     for ann in annotations:
#         cls, x_center, y_center, width, height = ann
#         resized_annotations.append([
#             cls,
#             x_center * scale_x,
#             y_center * scale_y,
#             width * scale_x,
#             height * scale_y
#         ])
#     return resized_annotations

# def calculate_overlap_area(x1, y1, x2, y2, size):
#     # Calculate the overlapping area between two squares
#     dx = min(x1 + size, x2 + size) - max(x1, x2)
#     dy = min(y1 + size, y2 + size) - max(y1, y2)
#     if dx >= 0 and dy >= 0:
#         return dx * dy
#     return 0

# def filter_crops(crops, threshold):
#     filtered_crops = []
#     for i, (img1, x1, y1) in enumerate(crops):
#         add_crop = True
#         for j, (img2, x2, y2) in enumerate(filtered_crops):
#             overlap_area = calculate_overlap_area(x1, y1, x2, y2, img1.size[0])
#             if overlap_area / (img1.size[0] * img1.size[1]) > threshold:
#                 add_crop = False
#                 break
#         if add_crop:
#             filtered_crops.append((img1, x1, y1))
#     return filtered_crops

# def process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, final_size, base_name, output_dir, images_with_objects, images_without_objects):
#     objects_in_crop = []
#     for ann in annotations:
#         cls, x_center, y_center, width, height = ann
#         x_center_abs = x_center * w
#         y_center_abs = y_center * h
#         width_abs = width * w
#         height_abs = height * h
        
#         if is_object_fully_contained(x_offset, y_offset, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
#             new_x_center = (x_center_abs - x_offset) / crop_size
#             new_y_center = (y_center_abs - y_offset) / crop_size
#             new_width = width_abs / crop_size
#             new_height = height_abs / crop_size
#             objects_in_crop.append([cls, new_x_center, new_y_center, new_width, new_height])
    
#     resized_img = img.resize((final_size, final_size), Image.LANCZOS)
#     if objects_in_crop:
#         resized_annotations = resize_annotations(objects_in_crop, (crop_size, crop_size), (final_size, final_size))
#         images_with_objects.append((resized_img, resized_annotations))
#     elif is_crop_without_objects(x_offset, y_offset, annotations, w, h, crop_size):
#         images_without_objects.append(resized_img)

# def crop_image(image, annotations, output_dir, base_name):
#     w, h = image.size

#     for crop_size in CROP_SIZES:
#         if h < crop_size or w < crop_size:
#             print("base_name", crop_size, base_name)
#             continue  # Пропустить, если изображение меньше текущего crop_size
        
#         cropped_images = []
        
#         # Нарезаем изображение на NUM_CROPS случайных фрагментов
#         while len(cropped_images) < NUM_CROPS:
#             x = random.randint(0, w - crop_size)
#             y = random.randint(0, h - crop_size)
#             cropped_img = image.crop((x, y, x + crop_size, y + crop_size))
#             cropped_images.append((cropped_img, x, y))
        
#         # Фильтруем фрагменты по критерию перекрытия
#         cropped_images = filter_crops(cropped_images, OVERLAP_THRESHOLD)
        
#         images_with_objects = []
#         images_without_objects = []
        
#         for img, x_offset, y_offset in cropped_images:
#             process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, FINAL_SIZE, base_name, output_dir, images_with_objects, images_without_objects)
        
#         selected_with_objects = random.sample(images_with_objects, min(NUM_WITH_OBJECTS, len(images_with_objects)))
#         max_without_objects = K * len(selected_with_objects)
#         selected_without_objects = random.sample(images_without_objects, min(max_without_objects, len(images_without_objects)))
        
#         for i, (img, objs) in enumerate(selected_with_objects):
#             img_name = f"{base_name}_{crop_size}_obj_{i}.jpg"
#             img.save(os.path.join(output_dir, img_name))
#             ann_name = f"{base_name}_{crop_size}_obj_{i}.txt"
#             write_yolo_annotations(os.path.join(output_dir, ann_name), objs, crop_size)
        
#         for i, img in enumerate(selected_without_objects):
#             img_name = f"{base_name}_{crop_size}_no-obj_{i}.jpg"
#             img.save(os.path.join(output_dir, img_name))

# def process_directory(image_dir, annotation_dir, output_dir):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
#     for image_file in image_files:
#         base_name = os.path.splitext(image_file)[0]
#         image_path = os.path.join(image_dir, image_file)
#         annotation_path = os.path.join(annotation_dir, base_name + '.txt')
        
#         if os.path.exists(annotation_path):
#             image = Image.open(image_path)
#             annotations = read_yolo_annotations(annotation_path)
#             crop_image(image, annotations, output_dir, base_name)

In [32]:
# import os
# import random
# from PIL import Image

# def read_yolo_annotations(file_path):
#     annotations = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             annotations.append([float(x) for x in line.strip().split()])
#     return annotations

# def write_yolo_annotations(file_path, annotations, crop_size):
#     with open(file_path, 'w') as file:
#         for ann in annotations:
#             koef = crop_size / FINAL_SIZE
#             ann = [int(ann[0]), koef*ann[1], koef*ann[2], koef*ann[3], koef*ann[4]]
#             file.write(' '.join(map(str, ann)) + '\n')

# def is_object_fully_contained(x, y, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
#     return (x < x_center_abs - width_abs / 2 and
#             x + crop_size > x_center_abs + width_abs / 2 and
#             y < y_center_abs - height_abs / 2 and
#             y + crop_size > y_center_abs + height_abs / 2)

# def is_crop_without_objects(x, y, annotations, w, h, crop_size):
#     for ann in annotations:
#         _, x_center, y_center, width, height = ann
#         x_center_abs = x_center * w
#         y_center_abs = y_center * h
#         width_abs = width * w
#         height_abs = height * h

#         if not (x + crop_size <= x_center_abs - width_abs / 2 or
#                 x >= x_center_abs + width_abs / 2 or
#                 y + crop_size <= y_center_abs - height_abs / 2 or
#                 y >= y_center_abs + height_abs / 2):
#             return False
#     return True

# def resize_annotations(annotations, original_size, target_size):
#     scale_x = target_size[0] / original_size[0]
#     scale_y = target_size[1] / original_size[1]
#     resized_annotations = []
#     for ann in annotations:
#         cls, x_center, y_center, width, height = ann
#         resized_annotations.append([
#             cls,
#             x_center * scale_x,
#             y_center * scale_y,
#             width * scale_x,
#             height * scale_y
#         ])
#     return resized_annotations

# def calculate_overlap_area(x1, y1, x2, y2, size):
#     dx = min(x1 + size, x2 + size) - max(x1, x2)
#     dy = min(y1 + size, y2 + size) - max(y1, y2)
#     if dx >= 0 and dy >= 0:
#         return dx * dy
#     return 0

# def filter_crops(crops, threshold):
#     filtered_crops = []
#     for i, (img1, x1, y1) in enumerate(crops):
#         add_crop = True
#         for j, (img2, x2, y2) in enumerate(filtered_crops):
#             overlap_area = calculate_overlap_area(x1, y1, x2, y2, img1.size[0])
#             if overlap_area / (img1.size[0] * img1.size[1]) > threshold:
#                 add_crop = False
#                 break
#         if add_crop:
#             filtered_crops.append((img1, x1, y1))
#     return filtered_crops

# def process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, final_size, base_name, output_dir, images_with_objects, images_without_objects):
#     objects_in_crop = []
#     for ann in annotations:
#         cls, x_center, y_center, width, height = ann
#         x_center_abs = x_center * w
#         y_center_abs = y_center * h
#         width_abs = width * w
#         height_abs = height * h
        
#         if is_object_fully_contained(x_offset, y_offset, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
#             new_x_center = (x_center_abs - x_offset) / crop_size
#             new_y_center = (y_center_abs - y_offset) / crop_size
#             new_width = width_abs / crop_size
#             new_height = height_abs / crop_size
#             objects_in_crop.append([cls, new_x_center, new_y_center, new_width, new_height])
    
#     resized_img = img.resize((final_size, final_size), Image.LANCZOS)
#     if objects_in_crop:
#         resized_annotations = resize_annotations(objects_in_crop, (crop_size, crop_size), (final_size, final_size))
#         images_with_objects.append((resized_img, resized_annotations))
#     elif is_crop_without_objects(x_offset, y_offset, annotations, w, h, crop_size):
#         images_without_objects.append(resized_img)

# def crop_image(image, annotations, output_dir, base_name):
#     w, h = image.size

#     # Проверяем, подходит ли изображение под основные размеры
#     fits_in_any_size = False
#     for crop_size in CROP_SIZES[:-1]:
#         if h >= crop_size and w >= crop_size:
#             fits_in_any_size = True
#             break
    
#     # Если изображение не подходит под размеры [1024, 640, 512], используем размер 256
#     if not fits_in_any_size:
#         crop_sizes_to_use = [CROP_SIZES[-1]]
#     else:
#         crop_sizes_to_use = CROP_SIZES[:-1]

#     for crop_size in crop_sizes_to_use:
#         if h < crop_size or w < crop_size:
#             continue  # Пропустить, если изображение меньше текущего crop_size
        
#         cropped_images = []
        
#         # Нарезаем изображение на NUM_CROPS случайных фрагментов
#         while len(cropped_images) < NUM_CROPS:
#             x = random.randint(0, w - crop_size)
#             y = random.randint(0, h - crop_size)
#             cropped_img = image.crop((x, y, x + crop_size, y + crop_size))
#             cropped_images.append((cropped_img, x, y))
        
#         # Фильтруем фрагменты по критерию перекрытия
#         cropped_images = filter_crops(cropped_images, OVERLAP_THRESHOLD)
        
#         images_with_objects = []
#         images_without_objects = []
        
#         for img, x_offset, y_offset in cropped_images:
#             process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, FINAL_SIZE, base_name, output_dir, images_with_objects, images_without_objects)
        
#         selected_with_objects = random.sample(images_with_objects, min(NUM_WITH_OBJECTS, len(images_with_objects)))
#         max_without_objects = K * len(selected_with_objects)
#         selected_without_objects = random.sample(images_without_objects, min(max_without_objects, len(images_without_objects)))
        
#         for i, (img, objs) in enumerate(selected_with_objects):
#             img_name = f"{base_name}_{crop_size}_obj_{i}.jpg"
#             img.save(os.path.join(output_dir, img_name))
#             ann_name = f"{base_name}_{crop_size}_obj_{i}.txt"
#             write_yolo_annotations(os.path.join(output_dir, ann_name), objs, crop_size)
        
#         for i, img in enumerate(selected_without_objects):
#             img_name = f"{base_name}_{crop_size}_no-obj_{i}.jpg"
#             img.save(os.path.join(output_dir, img_name))

# def process_directory(image_dir, annotation_dir, output_dir):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
#     for image_file in image_files:
#         base_name = os.path.splitext(image_file)[0]
#         image_path = os.path.join(image_dir, image_file)
#         annotation_path = os.path.join(annotation_dir, base_name + '.txt')
        
#         if os.path.exists(annotation_path):
#             image = Image.open(image_path)
#             annotations = read_yolo_annotations(annotation_path)
#             crop_image(image, annotations, output_dir, base_name)

# # Константы
# CROP_SIZES = [1024, 640, 512, 256]
# NUM_CROPS = 120
# NUM_WITH_OBJECTS = 4
# K = 1  # Максимальное отношение числа изображений без объектов к числу изображений с объектами
# FINAL_SIZE = 640
# OVERLAP_THRESHOLD = 0.5  # Максимальный процент перекрытия


In [52]:
import os
import random
from PIL import Image

def read_yolo_annotations(file_path):
    annotations = []
    with open(file_path, 'r') as file:
        for line in file:
            annotations.append([float(x) for x in line.strip().split()])
    return annotations

def write_yolo_annotations(file_path, annotations, crop_size):
    with open(file_path, 'w') as file:
        for ann in annotations:
            koef = crop_size / FINAL_SIZE
            ann = [int(ann[0]), koef*ann[1], koef*ann[2], koef*ann[3], koef*ann[4]]
            file.write(' '.join(map(str, ann)) + '\n')

def is_object_fully_contained(x, y, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
    return (x < x_center_abs - width_abs / 2 and
            x + crop_size > x_center_abs + width_abs / 2 and
            y < y_center_abs - height_abs / 2 and
            y + crop_size > y_center_abs + height_abs / 2)

def is_crop_without_objects(x, y, annotations, w, h, crop_size):
    for ann in annotations:
        _, x_center, y_center, width, height = ann
        x_center_abs = x_center * w
        y_center_abs = y_center * h
        width_abs = width * w
        height_abs = height * h

        if not (x + crop_size <= x_center_abs - width_abs / 2 or
                x >= x_center_abs + width_abs / 2 or
                y + crop_size <= y_center_abs - height_abs / 2 or
                y >= y_center_abs + height_abs / 2):
            return False
    return True

def resize_annotations(annotations, original_size, target_size):
    scale_x = target_size[0] / original_size[0]
    scale_y = target_size[1] / original_size[1]
    resized_annotations = []
    for ann in annotations:
        cls, x_center, y_center, width, height = ann
        resized_annotations.append([
            cls,
            x_center * scale_x,
            y_center * scale_y,
            width * scale_x,
            height * scale_y
        ])
    return resized_annotations

def calculate_overlap_area(x1, y1, x2, y2, size):
    dx = min(x1 + size, x2 + size) - max(x1, x2)
    dy = min(y1 + size, y2 + size) - max(y1, y2)
    if dx >= 0 and dy >= 0:
        return dx * dy
    return 0

def filter_crops(crops, threshold):
    filtered_crops = []
    for i, (img1, x1, y1) in enumerate(crops):
        add_crop = True
        for j, (img2, x2, y2) in enumerate(filtered_crops):
            overlap_area = calculate_overlap_area(x1, y1, x2, y2, img1.size[0])
            if overlap_area / (img1.size[0] * img1.size[1]) > threshold:
                add_crop = False
                break
        if add_crop:
            filtered_crops.append((img1, x1, y1))
    return filtered_crops

def process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, final_size, base_name, output_dir, images_with_objects, images_without_objects):
    objects_in_crop = []
    for ann in annotations:
        cls, x_center, y_center, width, height = ann
        x_center_abs = x_center * w
        y_center_abs = y_center * h
        width_abs = width * w
        height_abs = height * h
        
        if is_object_fully_contained(x_offset, y_offset, x_center_abs, y_center_abs, width_abs, height_abs, crop_size):
            new_x_center = (x_center_abs - x_offset) / crop_size
            new_y_center = (y_center_abs - y_offset) / crop_size
            new_width = width_abs / crop_size
            new_height = height_abs / crop_size
            objects_in_crop.append([cls, new_x_center, new_y_center, new_width, new_height])
    
    resized_img = img.resize((final_size, final_size), Image.LANCZOS)
    if objects_in_crop:
        resized_annotations = resize_annotations(objects_in_crop, (crop_size, crop_size), (final_size, final_size))
        images_with_objects.append((resized_img, resized_annotations))
    elif is_crop_without_objects(x_offset, y_offset, annotations, w, h, crop_size):
        images_without_objects.append(resized_img)

def crop_image(image, annotations, output_dir, base_name):
    w, h = image.size

    fits_in_any_size = False
    for crop_size in CROP_SIZES[:-1]:
        if h >= crop_size and w >= crop_size:
            fits_in_any_size = True
            break
    
    if not fits_in_any_size:
        crop_sizes_to_use = [CROP_SIZES[-1]]
    else:
        crop_sizes_to_use = CROP_SIZES[:-1]

    for crop_size in crop_sizes_to_use:
        if h < crop_size or w < crop_size:
            continue 
        
        cropped_images = []

        
        while len(cropped_images) < NUM_CROPS:
            x = random.randint(0, w - crop_size)
            y = random.randint(0, h - crop_size)
            cropped_img = image.crop((x, y, x + crop_size, y + crop_size))
            cropped_images.append((cropped_img, x, y))

        
        cropped_images = filter_crops(cropped_images, OVERLAP_THRESHOLD)
        
        images_with_objects = []
        images_without_objects = []
        
        for img, x_offset, y_offset in cropped_images:
            process_crop(img, annotations, crop_size, x_offset, y_offset, w, h, FINAL_SIZE, base_name, output_dir, images_with_objects, images_without_objects)
        
        selected_with_objects = random.sample(images_with_objects, min(NUM_WITH_OBJECTS, len(images_with_objects)))
        max_without_objects = K * len(selected_with_objects)
        selected_without_objects = random.sample(images_without_objects, min(max_without_objects, len(images_without_objects)))
        
        for i, (img, objs) in enumerate(selected_with_objects):
            img_name = f"{base_name}_{crop_size}_obj_{i}.jpg"
            img.save(os.path.join(output_dir, img_name))
            ann_name = f"{base_name}_{crop_size}_obj_{i}.txt"
            write_yolo_annotations(os.path.join(output_dir, ann_name), objs, crop_size)
        
        for i, img in enumerate(selected_without_objects):
            img_name = f"{base_name}_{crop_size}_no-obj_{i}.jpg"
            img.save(os.path.join(output_dir, img_name))

    
    if not fits_in_any_size:
        resized_img = image.resize((FINAL_SIZE, FINAL_SIZE), Image.LANCZOS)
        resized_annotations = resize_annotations(annotations, (w, h), (FINAL_SIZE, FINAL_SIZE))
        img_name = f"{base_name}_original.jpg"
        ann_name = f"{base_name}_original.txt"
        resized_img.save(os.path.join(output_dir, img_name))
        write_yolo_annotations(os.path.join(output_dir, ann_name), resized_annotations, FINAL_SIZE)

def process_directory(image_dir, annotation_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(image_dir, image_file)
        annotation_path = os.path.join(annotation_dir, base_name + '.txt')
        
        if not os.path.exists(annotation_path):
            open(annotation_path, 'w').close()

        if os.path.exists(annotation_path):
            image = Image.open(image_path)
            annotations = read_yolo_annotations(annotation_path)
            crop_image(image, annotations, output_dir, base_name)


CROP_SIZES = [1024, 640, 512, 256]
NUM_CROPS = 120
NUM_WITH_OBJECTS = 4
K = 1 
FINAL_SIZE = 640
OVERLAP_THRESHOLD = 0.5 



In [53]:
CROP_SIZES = [1024, 640, 512, 256]
NUM_CROPS = 120
NUM_WITH_OBJECTS = 4
K = 1 
FINAL_SIZE = 640
OVERLAP_THRESHOLD = 0.5 



In [39]:

# image_path = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/dataset_mmdet/test_new/'
# annotations_path = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/dataset_mmdet/test_new/'
# output_dir = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/dataset_mmdet/test_new_cropped'

image_path = '/Users/elinachertova/Downloads/NO_BPLA/'
annotations_path = '/Users/elinachertova/Downloads/NO_BPLA/'
output_dir = '/Users/elinachertova/Downloads/NO_BPLA_crop'
process_directory(image_path, annotations_path, output_dir)

In [ ]:
train_cropped_with_small

In [15]:
import os


len(os.listdir('/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/train_cropped_cl2'))

104762

Удалено: 27_1285.txt 1
Удалено: 45_137.txt 2
Удалено: 91_1788.txt 3
Завершено.


In [45]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pandas as pd

In [46]:
def load_yolo_annotations(file_path):
    annotations = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            try:
                label = int(float(parts[0]))
                x_center, y_center, width, height = map(float, parts[1:])
                annotations.append((label, x_center, y_center, width, height))
            except ValueError as e:
                print(f"Error parsing line '{line.strip()}': {e}")
    return annotations

def plot_image_with_annotations(image_path, annotations, ax):
    image = Image.open(image_path)
    ax.imshow(image)
    for annotation in annotations:
        label, x_center, y_center, width, height = annotation
        img_width, img_height = image.size
        rect = patches.Rectangle(
            ((x_center - width / 2) * img_width, (y_center - height / 2) * img_height),
            width * img_width, height * img_height,
            linewidth=1, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text((x_center - width / 2) * img_width, (y_center - height / 2) * img_height, str(label), color='white', backgroundcolor='red')

def display_images_with_annotations(folder_path, batch_size=2):
    images = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    total_images = len(images)
    
    for i in range(0, total_images, batch_size):
        fig, axs = plt.subplots(1, 2, figsize=(20, 10))
        axs = axs.flatten()
        for j in range(batch_size):
            idx = i + j
            if idx < total_images:
                image_file = images[idx]
                image_path = os.path.join(folder_path, image_file)
                annotation_file = os.path.splitext(image_file)[0] + ".txt"
                annotation_path = os.path.join(folder_path, annotation_file)
                if os.path.exists(annotation_path):
                    annotations = load_yolo_annotations(annotation_path)
                else:
                    annotations = []
                plot_image_with_annotations(image_path, annotations, axs[j])
            axs[j].axis('off')
        plt.show()

In [2]:

# folder_path = 'path_to_your_folder'
output_dir = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/dataset_mmdet/test_new_cropped'
display_images_with_annotations(output_dir)

In [1]:
import os
import cv2
import numpy as np
from glob import glob
from collections import defaultdict
import shutil


data_dir = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/dataset_mmdet/test_cropped_last'
destination_dir = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/blue_sky_no_annotations_test_smal'

os.makedirs(destination_dir, exist_ok=True)
image_extensions = ['*.jpg', '*.jpeg', '*.png']

image_files = []
for ext in image_extensions:
    image_files.extend(glob(os.path.join(data_dir, ext)))

def is_blue_sky(image_path):
    """Check if the image is predominantly blue sky."""
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])

    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    
    blue_ratio = np.sum(mask > 0) / (image.shape[0] * image.shape[1])
    
    return blue_ratio > 0.8

def get_base_name(image_path):
    filename = os.path.basename(image_path)
    base_name = '_'.join(filename.split('_')[:-2])
    return base_name
    
image_groups = defaultdict(list)
for image_file in image_files:
    base_name = get_base_name(image_file)
    print("base_name", base_name)
    image_groups[base_name].append(image_file)


for base_name, images in image_groups.items():
    no_annotation_images = []
    
    for image_file in images:
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        
        if not os.path.exists(annotation_file):
            if is_blue_sky(image_file):
                no_annotation_images.append(image_file)

    
    if len(no_annotation_images) > 1:
        for image_to_move in no_annotation_images[1:]:
            destination_path = os.path.join(destination_dir, os.path.basename(image_to_move))
            print(f"Moving image without annotation: {image_to_move} to {destination_path}")
            shutil.move(image_to_move, destination_path)

print("Cleanup completed.")


In [5]:
# import os
# import cv2
# import numpy as np
# from glob import glob
# from collections import defaultdict

# # Path to the directory containing images and annotations
# data_dir = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/train_cropped'

# # Supported image file extensions
# image_extensions = ['*.jpg', '*.jpeg', '*.png']

# # Get a list of all image files with supported extensions
# image_files = []
# for ext in image_extensions:
#     image_files.extend(glob(os.path.join(data_dir, ext)))

# def is_blue_sky(image_path):
#     """Check if the image is predominantly blue sky."""
#     image = cv2.imread(image_path)
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

#     # Define range for blue color in HSV
#     lower_blue = np.array([90, 50, 50])
#     upper_blue = np.array([130, 255, 255])

#     # Create a mask for blue color
#     mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    
#     # Calculate the percentage of blue pixels
#     blue_ratio = np.sum(mask > 0) / (image.shape[0] * image.shape[1])
    
#     # If more than 80% of the image is blue, consider it as blue sky
#     return blue_ratio > 0.8

# # Function to get the base name of the image (excluding the unique suffix)
# def get_base_name(image_path):
#     filename = os.path.basename(image_path)
#     base_name = '_'.join(filename.split('_')[:-2])
#     print("base_name", base_name)
#     return base_name

# # Group images by their base names
# image_groups = defaultdict(list)
# for image_file in image_files:
#     base_name = get_base_name(image_file)
#     image_groups[base_name].append(image_file)

# # Process each group
# for base_name, images in image_groups.items():
#     no_annotation_images = []
    
#     for image_file in images:
#         annotation_file = os.path.splitext(image_file)[0] + '.txt'
        
#         # Check if annotation file exists
#         if not os.path.exists(annotation_file):
#             # Check if the image is predominantly blue sky
#             if is_blue_sky(image_file):
#                 no_annotation_images.append(image_file)
    
#     # If there are multiple blue sky images without annotations, delete all but one
#     if len(no_annotation_images) > 1:
#         # Keep the first one and delete the rest
#         for image_to_delete in no_annotation_images[1:]:
#             print(f"Deleting image without annotation: {image_to_delete}")
#             # os.remove(image_to_delete)

# print("Cleanup completed.")


In [ ]:
import os
import random
from PIL import Image

def crop_image(image, output_dir, base_name):
    w, h = image.size

    fits_in_any_size = False
    for crop_size in CROP_SIZES[:-1]:
        if h >= crop_size and w >= crop_size:
            fits_in_any_size = True
            break
    
    if not fits_in_any_size:
        crop_sizes_to_use = [CROP_SIZES[-1]]
    else:
        crop_sizes_to_use = CROP_SIZES[:-1]

    for crop_size in crop_sizes_to_use:
        if h < crop_size or w < crop_size:
            continue  
            
        cropped_images = []
        
        while len(cropped_images) < NUM_CROPS:
            x = random.randint(0, w - crop_size)
            y = random.randint(0, h - crop_size)
            cropped_img = image.crop((x, y, x + crop_size, y + crop_size))
            cropped_images.append((cropped_img, x, y))
        
        cropped_images = filter_crops(cropped_images, OVERLAP_THRESHOLD)
        
        for i, (img, x_offset, y_offset) in enumerate(cropped_images):
            resized_img = img.resize((FINAL_SIZE, FINAL_SIZE), Image.LANCZOS)
            img_name = f"{base_name}_{crop_size}_crop_{i}.jpg"
            resized_img.save(os.path.join(output_dir, img_name))

    if not fits_in_any_size:
        resized_img = image.resize((FINAL_SIZE, FINAL_SIZE), Image.LANCZOS)
        img_name = f"{base_name}_original.jpg"
        resized_img.save(os.path.join(output_dir, img_name))

def filter_crops(crops, threshold):
    filtered_crops = []
    for i, (img1, x1, y1) in enumerate(crops):
        add_crop = True
        for j, (img2, x2, y2) in enumerate(filtered_crops):
            overlap_area = calculate_overlap_area(x1, y1, x2, y2, img1.size[0])
            if overlap_area / (img1.size[0] * img1.size[1]) > threshold:
                add_crop = False
                break
        if add_crop:
            filtered_crops.append((img1, x1, y1))
    return filtered_crops

def calculate_overlap_area(x1, y1, x2, y2, size):
    dx = min(x1 + size, x2 + size) - max(x1, x2)
    dy = min(y1 + size, y2 + size) - max(y1, y2)
    if dx >= 0 and dy >= 0:
        return dx * dy
    return 0

def process_directory(image_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(image_dir, image_file)
        image = Image.open(image_path)
        crop_image(image, output_dir, base_name)


CROP_SIZES = [1024, 640, 512, 256]
NUM_CROPS = 20
FINAL_SIZE = 640
OVERLAP_THRESHOLD = 0.5  


image_dir = '/Users/elinachertova/Downloads/NO_BPLA' 
output_dir = 'Users/elinachertova/Downloads/NO_BPLA_сrop' 

process_directory(image_dir, output_dir)


In [13]:
import os

len(os.listdir("/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/train_cropped"))

252368

In [14]:
import os

folder_path = '/Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/train_cropped'


image_extensions = ('.jpg', '.jpeg', '.png')

def count_images(folder_path, extensions):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(extensions):
            count += 1
    return count


image_count = count_images(folder_path, image_extensions)

print(f"Количество изображений в папке {folder_path}: {image_count}")


Количество изображений в папке /Users/elinachertova/Downloads/hackathon_additional_dataset/dataset/train_cropped: 157993


In [ ]:
od.